In [1]:
!pip freeze


aiohttp==3.8.4
aiosignal==1.3.1
altair==5.0.1
asttokens==2.2.1
async-timeout==4.0.2
attrs==23.1.0
backcall==0.2.0
blinker==1.6.2
cachetools==5.3.1
certifi==2023.5.7
cffi==1.15.1
chardet==5.1.0
charset-normalizer==3.1.0
click==8.1.3
colorama==0.4.6
comm==0.1.3
cryptography==41.0.2
dataclasses-json==0.5.8
debugpy==1.6.7
decorator==5.1.1
dnspython==2.3.0
et-xmlfile==1.1.0
executing==1.2.0
filelock==3.12.2
filetype==1.2.0
frozenlist==1.3.3
fsspec==2023.6.0
gitdb==4.0.10
GitPython==3.1.31
greenlet==2.0.2
huggingface-hub==0.16.4
idna==3.4
importlib-metadata==6.7.0
ipykernel==6.23.2
ipython==8.14.0
jedi==0.18.2
Jinja2==3.1.2
joblib==1.3.1
jsonschema==4.17.3
jupyter_client==8.2.0
jupyter_core==5.3.1
langchain==0.0.205
langchainplus-sdk==0.0.13
loguru==0.7.0
lxml==4.9.3
Markdown==3.4.4
markdown-it-py==3.0.0
MarkupSafe==2.1.3
marshmallow==3.19.0
marshmallow-enum==1.5.1
matplotlib-inline==0.1.6
mdurl==0.1.2
mpmath==1.3.0
msg-parser==1.2.0
multidict==6.0.4
mypy-extensions==1.0.0
nest-asyncio==1.5.

In [1]:
import os
import tempfile
import streamlit as st
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import openai
from langchain.llms.openai import OpenAI
import pinecone

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
file_path = f'C:/Users/user/Desktop/AES automatic electronic Slide user guide.pdf'


In [3]:
loader = UnstructuredPDFLoader(file_path)
#pages2 = loader2.load_and_split()
pages = loader.load()


In [5]:
#new_pdf2 = pages2[26:31]
#new_pdf2
pages

[Document(page_content="AES automatic electronic Slide Door Design and function description\n\nDesign: The sliding door operator Besam SL500 T works electromechanically. The motor, control unit, transmission – and optional emergency unit and electromechanical locking device – are all assembled in a support beam with an integrated cover. The motor and gear box transmit movement to the door leaves by means of a tooth belt. The door leaf is fitted to a door adapter/carriage wheel fitting and hangs on a sliding track. The guiding at the bottom is carried out by means of floor guides (Full Break-Out) or Side Panel Guides (Fixed Sidelites).\n\nFunction:\n\nOpening When an OPENING IMPULSE is received by the control unit the motor starts and transmits movement to the door leaves, which move to the open position.\n\nClosing The closing starts when no OPENING IMPULSE is received and the HOLD OPEN TIME has expired.\n\nSafety functions integrated in the operator To permit safe passage between clos

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
doc_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 50)
new_pdf2 = doc_splitter.split_documents(pages)
len(new_pdf2)

36

In [9]:
os.environ["OPENAI_API_KEY"] = "sk-xxx"
pinecone.init(api_key="xxx", environment="northamerica-northeast1-gcp")

In [10]:
# Create embeddings for the pages and insert into pinecone database
embeddings = OpenAIEmbeddings()
index_name = "marketing-pr"
pinecone.create_index(index_name, dimension=1536, metric="cosine")
index = pinecone.Index(index_name=index_name)


In [11]:
from langchain.vectorstores import Pinecone
# Create document search database
db =  Pinecone.from_documents(new_pdf2, embeddings, index_name=index_name)


In [21]:
db = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [12]:
retriever = db.as_retriever()
# Define the template for the language model prompt
template =  """
            You are a virtual technician designed to provide support and information to clients on automatic sliding door systems.\
            Your objective is to provide accurate and simplified responses to a wide range of \
            automatic sliding door systems questions, based on your knowledge base. \

            If a user's query indicates an error code or number, go through document to give possible reasons and remedies to the issue\
            

            In your responses, ensure a tone of professionalism, understanding and expertise. Provide users with \
            possible solutions and stop gaps to their issues pending when the physical technician will arrive \
        

            Here are some specific interaction scenarios to guide your responses:
            - If the- user asks what you can do, respond with "I'm a virtual technician here to provide \
            first aid support and information on the technical issue you are experiencing. How can I assist you?"
            - If the user starts with a greeting, respond with 'Hello! How are you doing today? How can I assist you?' \
            or something related to that
            - If a user shares their name, use it in your responses when appropriate, to cultivate a more personal and \
            comforting conversation.

            - If the exact question is not available, ask the user to reboot the system and then provide a suggestion based on the second nearest answer from the pdf.\
            - If a user asks a question that is unrelated to automated systems, respond with \
            'This question is out of my scope as I'm built mainly as a first aid support for our electronic systems in your facility \
            Could you please ask a question related to our automated systems?'

            {context}
            Question: {question}
            Answer:"""

# Create a prompt template
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [13]:
chain_type_kwargs = {"prompt": prompt}
#Initialize the OpenAI module, load and run the retrieval QA chain
llm = OpenAI(temperature=0)
chain = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
chain_type_kwargs=chain_type_kwargs,
verbose=True
)

In [24]:
user_question = input("How can we assist you today?")
user_question

'my door is stuck'

In [25]:
#search = db.similarity_search(user_question)
answer = chain.run(user_question)
answer



> Entering new  chain...

> Finished chain.


' It sounds like you are having an issue with your door being stuck. This could be due to a number of reasons, such as sand, stones, rubbish, etc. in the floor guide, or the door leaf jamming on the floor or on the sweeps. I recommend that you disconnect the mains power and unlock all mechanical locks, then pull the door leaf manually and check that the door can be easily moved over the complete sliding track/floor guide. If the door leaf stops or is hard to move, you should clean the floor guide, adjust the door leaf height/depth or take other necessary measures, such as replacing any worn parts.'